IMPORT PACKAGES

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import pandas as pd
import numpy as np
from tensorflow import keras
from tqdm import tqdm
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from nltk.tokenize import word_tokenize

from string import punctuation
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras import utils
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


INISIALISASI PATH

In [ ]:
train = pd.read_excel('/content/drive/MyDrive/DATASET/DATA-FINAL/Data-train_sentimen_covid.xlsx' )
test  = pd.read_excel('/content/drive/MyDrive/DATASET/DATA-FINAL/Data-dev_sentimen_covid.xlsx')
validation = pd.read_excel('/content/drive/MyDrive/DATASET/DATA-FINAL/Data-test_sentimen_covid.xlsx')

In [ ]:
pos = train[train.Sentiment == 'pos'] 
net = train[train.Sentiment == 'net'][:1200] 
neg = train[train.Sentiment == 'neg'] 
train_new = pd.concat([pos,net,neg])

In [ ]:
train_new["Sentiment"].value_counts(normalize=True)

net    0.473186
neg    0.344243
pos    0.182571
Name: Sentiment, dtype: float64

In [ ]:
train = train
train.count()

Unnamed: 0    8000
Sentiment     8000
Tweet         8000
dtype: int64

In [ ]:
validation.count()

Unnamed: 0    1000
Sentiment     1000
Tweet         1000
dtype: int64

In [ ]:
pip install bs4

In [ ]:
pip install sastrawi

     |████████████████████████████████| 209 kB 29.6 MB/s 


In [ ]:
import string
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

TEXT PREPROCESSING

In [ ]:

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer 
import re
lm=WordNetLemmatizer()

def pre_process(text):
    # Case Folding: Lowercase
    # Merubah format teks menjadi format huruf kecil semua (lowercase).
    text = text.lower()

    # Case Folding: Removing Number
    # Menghapus karakter angka.
    text = re.sub(r"\d+", "", text)

    # Case Folding: Removing Punctuation
    # Menghapus karakter tanda baca.
    text = text.translate(str.maketrans("","",string.punctuation))

    #Case Folding: Removing Whitespace
    #Menghapus karakter kosong.
    text = text.strip()

    
    #Separating Sentences with Split () Method
    #Fungsi split() memisahkan string ke dalam list dengan spasi sebagai pemisah jika tidak ditentukan pemisahnya.
    pisah = text.split()

    #Filtering using sastrawi
    factory = StopWordRemoverFactory()
    stopword = factory.create_stop_word_remover()
    text =  stopword.remove(text)

    return text

train['Tweet'] = train['Tweet'].apply(lambda x:pre_process(x))

train.count()

Unnamed: 0    8000
Sentiment     8000
Tweet         8000
dtype: int64

In [ ]:
train_new['Tweet']=train_new['Tweet'].apply(lambda x:pre_process(x))

In [ ]:
train_new.count()

Unnamed: 0    2536
Sentiment     2536
Tweet         2536
dtype: int64

In [ ]:
test['Tweet']=test['Tweet'].apply(lambda x:pre_process(x))

In [ ]:
validation['Tweet']=validation['Tweet'].apply(lambda x:pre_process(x))

In [ ]:
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer 
import re
lm=WordNetLemmatizer()
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras import utils
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Bidirectional, Dropout


TOKENISASI DATA TRAIN

In [ ]:
max_features = 2000
tokenizer = Tokenizer(num_words = max_features, )
tokenizer.fit_on_texts(train_new['Tweet'].values)
X = tokenizer.texts_to_sequences(train_new['Tweet'].values)
X = pad_sequences(X, padding = 'post' ,maxlen=300)
Y = tokenizer.texts_to_sequences(test['Tweet'].values)
Y = pad_sequences(Y, padding = 'post' ,maxlen=300)
Y_train = pd.get_dummies(train_new['Sentiment']).values
Y_test = pd.get_dummies(test['Sentiment']).values

V = tokenizer.texts_to_sequences(validation['Tweet'].values)
V = pad_sequences(V, padding = 'post' ,maxlen=300)
V_test = pd.get_dummies(validation['Sentiment']).values
vocab_size = len(tokenizer.word_index)+1

V_train=V
X_train=X
X_test=Y

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)
print(V_train.shape)
print(V_test.shape)

(2536, 300)
(778, 300)
(2536, 3)
(778, 3)
(1000, 300)
(1000, 3)


In [ ]:
from tqdm import tqdm
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

MEMBUAT KORPUS WORD2VEC UNTUK WORD EMBEDDING DENGAN DATA TRAIN LAMA

In [ ]:
sentences =[]
for t in  tqdm(range(len(train['Tweet']))):
    text = nltk.word_tokenize(train['Tweet'][t])
    sentences.append(text)

100%|██████████| 8000/8000 [00:01<00:00, 6103.25it/s]


In [ ]:
from gensim.models import Word2Vec
w2v_model = Word2Vec(sentences, size=300, min_count=2, sg = 0 )

In [ ]:
words = list(w2v_model.wv.vocab)
print('Vocabulary size: %d' % len(words))

# save model 
filename = 'embedding_word2vec.txt'
w2v_model.wv.save_word2vec_format(filename, binary=False)

Vocabulary size: 7086


In [ ]:
embedding_vector = {}
f = open('./embedding_word2vec.txt')
for line in tqdm(f):
    value = line.split(' ')
    word = value[0]
    coef = np.array(value[1:],dtype = 'float32')
    embedding_vector[word] = coef

7087it [00:00, 10925.95it/s]


In [ ]:
embedding_matrix = np.zeros((vocab_size,300))
for word,i in tqdm(tokenizer.word_index.items()):
    embedding_value = embedding_vector.get(word)
    if embedding_value is not None:
        embedding_matrix[i] = embedding_value 

100%|██████████| 10139/10139 [00:00<00:00, 564590.00it/s]


ARSITEKTUR LSTM DENGAN MENGGUNAKAN WORD2VEC SEBAGAI WORD EMBEDDING✅

In [ ]:
embid_dim = 300
lstm_out = 128


model = keras.Sequential()
model.add(Embedding(vocab_size, embid_dim, input_length =X.shape[1], weights = [ embedding_matrix] , trainable = True))
model.add(LSTM(lstm_out, dropout=0.2))
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(3, activation = 'softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 300)          3042000   
                                                                 
 lstm (LSTM)                 (None, 128)               219648    
                                                                 
 dense (Dense)               (None, 128)               16512     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 3)                 195       
                                                                 
Total params: 3,286,611
Trainable params: 3,286,611
Non-

In [ ]:
es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
batch_size = 64
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
history = model.fit(X_train, Y_train, epochs = 100, batch_size=batch_size, verbose = 1, validation_data =(X_test, Y_test))

Epoch 1/50
40/40 [==============================] - 56s 1s/step - loss: 1.0334 - accuracy: 0.4732 - val_loss: 0.8369 - val_accuracy: 0.8329
Epoch 2/50
40/40 [==============================] - 52s 1s/step - loss: 1.0317 - accuracy: 0.4732 - val_loss: 0.8319 - val_accuracy: 0.8329
Epoch 3/50
40/40 [==============================] - 52s 1s/step - loss: 1.0347 - accuracy: 0.4732 - val_loss: 0.8329 - val_accuracy: 0.8329
Epoch 4/50
40/40 [==============================] - 54s 1s/step - loss: 1.0344 - accuracy: 0.4732 - val_loss: 0.8358 - val_accuracy: 0.8329
Epoch 5/50
40/40 [==============================] - 52s 1s/step - loss: 1.0344 - accuracy: 0.4732 - val_loss: 0.8387 - val_accuracy: 0.8329
Epoch 6/50
40/40 [==============================] - 52s 1s/step - loss: 1.0347 - accuracy: 0.4732 - val_loss: 0.8360 - val_accuracy: 0.8329
Epoch 7/50
40/40 [==============================] - 52s 1s/step - loss: 1.0323 - accuracy: 0.4732 - val_loss: 0.7973 - val_accuracy: 0.8329
Epoch 8/50
40/40 [==

In [ ]:
_, train_acc = model.evaluate(X_train, Y_train,verbose=0)

In [ ]:
_, validate_acc = model.evaluate(V_train, V_test, verbose=0)

In [ ]:
print(validate_acc)
print(train_acc)

0.75
0.9064748287200928


PRETRAINED WORD2VEC

In [ ]:
import gensim
model_pre = gensim.models.Word2Vec.load("/content/drive/MyDrive/model/Word2Vec/idwiki_word2vec_300.model") 

In [ ]:
embedding_matrix_pre = np.zeros((vocab_size,300))
for word,i in tqdm(tokenizer.word_index.items()):
    try:
        embedding_value_pre = model_pre[word]
        if embedding_value_pre is not None:
            embedding_matrix[i] = embedding_value_pre         
    except KeyError:
        embedding_matrix_pre[i]=np.random.normal(0,np.sqrt(0.25),300) 

  0%|          | 0/12693 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.
100%|██████████| 12693/12693 [00:00<00:00, 44909.90it/s]


In [ ]:
embid_dim = 300
lstm_out = 128


model_pre = keras.Sequential()
model_pre.add(Embedding(vocab_size, 300, input_length =300, weights = [embedding_matrix_pre ] , trainable = True))
model_pre.add(Bidirectional(LSTM(lstm_out, dropout=0.2)))
model_pre.add(Dense(128, activation = 'relu'))
model_pre.add(Dropout(0.5))
model_pre.add(Dense(64, activation = 'relu'))
model_pre.add(Dense(3, activation = 'softmax'))
model_pre.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 300, 300)          3808200   
                                                                 
 bidirectional_1 (Bidirectio  (None, 256)              439296    
 nal)                                                            
                                                                 
 dense_3 (Dense)             (None, 128)               32896     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dense_5 (Dense)             (None, 3)                 195       
                                                      

In [ ]:
es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
batch_size = 128
model_pre.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
history = model_pre.fit(X_train, Y_train, epochs = 50, batch_size=batch_size, verbose = 1, validation_data =(X_test, Y_test),callbacks=es_callback)

Epoch 1/50
27/27 [==============================] - 108s 4s/step - loss: 0.9764 - accuracy: 0.5818 - val_loss: 0.7461 - val_accuracy: 0.8329
Epoch 2/50
27/27 [==============================] - 102s 4s/step - loss: 0.8828 - accuracy: 0.6058 - val_loss: 0.6433 - val_accuracy: 0.8342
Epoch 3/50
27/27 [==============================] - 102s 4s/step - loss: 0.7320 - accuracy: 0.6754 - val_loss: 0.5249 - val_accuracy: 0.8213
Epoch 4/50
27/27 [==============================] - 103s 4s/step - loss: 0.5439 - accuracy: 0.7719 - val_loss: 0.5709 - val_accuracy: 0.7532
Epoch 5/50
27/27 [==============================] - 102s 4s/step - loss: 0.4390 - accuracy: 0.8237 - val_loss: 0.6354 - val_accuracy: 0.7404
Epoch 6/50
27/27 [==============================] - 101s 4s/step - loss: 0.3502 - accuracy: 0.8678 - val_loss: 0.7299 - val_accuracy: 0.7121


In [ ]:
_, train_acc_pre = model_pre.evaluate(X_train, Y_train,verbose=0)

In [ ]:
_, validate_acc_pre = model_pre.evaluate(V_train, V_test, verbose=0)

In [ ]:
print(train_acc_pre)
print(validate_acc_pre)

0.9121702909469604
0.7260000109672546
